In [3]:
import pandas as pd
import pandas.io.sql
import pymssql as sapsql 
import numpy as np
from IPython.display import display
from jinja2 import Environment, FileSystemLoader
import pdfkit 


env = Environment(loader=FileSystemLoader('/data/jupyter/'))
#template = env.get_template("template.html")

datacompany = ( {'host':'192.168.250.40' , 'dbname':'IGU_LIVE_20221204','user':'sa','password':'B1admin'},
                {'host':'192.168.250.40' , 'dbname':'PTCKI','user':'sa','password':'B1admin'},
                {'host':'192.168.250.40' , 'dbname':'PTSCA','user':'sa','password':'B1admin'},
                {'host':'192.168.250.40' , 'dbname':'PTBWN','user':'sa','password':'B1admin'},
                {'host':'192.168.250.40' , 'dbname':'PTIMS','user':'sa','password':'B1admin'},
              {'host':'192.168.250.40' , 'dbname':'IGU_NEW','user':'sa','password':'B1admin'},)

datalist=[]
 
for company in datacompany:
    msg_sql= """

select 
 '""" + company["dbname"] + """' Company,  
 a.docnum,
 a.docstatus,
 a.docdate,
 a.docduedate ETA,
 a.cardcode,
 a.cardname,
 b.itemcode,
 c.U_group,
 e.groupname,
 b.Dscription,
 b.linestatus,
 b.targettype,
 case 
  when b.targettype = 20 then 'GRPO'
  when b.targettype = 204 then 'A/P DP'
  else 'A/P Invoice' 
  end status,
 b.Quantity,
 b.OpenQty,
 b.unitmsr,
 b.currency,
 b.rate,
 a.doccur,
 a.docrate,
 b.price,
 b.VATgroup,
 b.VATsum,
 
 b.linetotal,
 case 
  when b.currency = 'IDR' then b.Openqty* a.docrate
  else b.openqty*b.rate
  end sisa
 
from opor a 
    left outer join por1 b on a.DocEntry = b.DocEntry
    left outer join oitm c on b.itemcode = c.itemcode
    left outer join ocrd d on a.cardcode = d.cardcode
    left outer join ocrg e on d.groupcode = e.groupcode

where 
    left(convert(varchar, a.DocDate,112),8) between '20220101' and '20230123' and a.canceled = 'N' and b.targettype != '18'
    
                """

    conn = sapsql.connect(host=company["host"] , user=company["user"] , password=company["password"], database=company["dbname"] )
    cursor = conn.cursor()     
    data = pandas.io.sql.read_sql(msg_sql,conn)
    datalist.append(data)

df = pd.concat(datalist)  
options = {
            'page-size': 'legal',
            'orientation': 'landscape',
            }

textstring = ""

df.info()
df

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  This is separate from the ipykernel package so we can avoid doing imports until


<class 'pandas.core.frame.DataFrame'>
Int64Index: 13067 entries, 0 to 590
Data columns (total 26 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Company     13067 non-null  object        
 1   docnum      13067 non-null  int64         
 2   docstatus   13067 non-null  object        
 3   docdate     13067 non-null  datetime64[ns]
 4   ETA         13067 non-null  datetime64[ns]
 5   cardcode    13067 non-null  object        
 6   cardname    13067 non-null  object        
 7   itemcode    13067 non-null  object        
 8   U_group     13062 non-null  object        
 9   groupname   13067 non-null  object        
 10  Dscription  13067 non-null  object        
 11  linestatus  13067 non-null  object        
 12  targettype  13067 non-null  int64         
 13  status      13067 non-null  object        
 14  Quantity    13067 non-null  float64       
 15  OpenQty     13067 non-null  float64       
 16  unitmsr     12908 non-nu

,Company,docnum,docstatus,docdate,ETA,cardcode,cardname,itemcode,U_group,groupname,...,unitmsr,currency,rate,doccur,docrate,price,VATgroup,VATsum,linetotal,sisa
0,IGU_LIVE_20221204,2090000093,C,2022-09-05,2022-09-21,VI0068,MULWARRA EXPORT PTY LTD,AAU1BLA001,A-MEAT,Import,...,KG,USD,14854.0,USD,14854.0,135.700,PPNM_00,0.000000e+00,1.340440e+07,0.0
1,IGU_LIVE_20221204,2090000093,C,2022-09-05,2022-09-21,VI0068,MULWARRA EXPORT PTY LTD,AAU1BLA002,A-MEAT,Import,...,KG,USD,14854.0,USD,14854.0,131.000,PPNM_00,0.000000e+00,1.663722e+07,0.0
2,IGU_LIVE_20221204,2090000168,C,2022-09-16,2022-10-13,VI0069,NH FOODS AUSTRALIA PTY LTD,AAU1BOR006,A-MEAT,Import,...,KG,USD,14897.0,USD,14897.0,21.000,PPNM_00,0.000000e+00,1.689007e+08,0.0
3,IGU_LIVE_20221204,2100000061,C,2022-10-03,2022-10-19,VI0069,NH FOODS AUSTRALIA PTY LTD,AAU1BOR006,A-MEAT,Import,...,KG,USD,15007.0,USD,15007.0,21.000,PPNM_00,0.000000e+00,1.711437e+08,0.0
4,IGU_LIVE_20221204,2080000236,O,2022-08-08,2022-09-28,VI0069,NH FOODS AUSTRALIA PTY LTD,AAU1BOR006,A-MEAT,Import,...,KG,USD,14963.0,USD,14963.0,21.000,PPNM_00,0.000000e+00,1.565648e+08,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,IGU_NEW,301100095,C,2023-01-23,2023-01-25,VL0161,"ALKALINDO ALAM ANUGRAH, PT",IID3ORG002,I-BEVERAGES,Lokal,...,BTL,IDR,0.0,IDR,1.0,10984.985,PPNM01,1.450018e+06,1.318198e+07,0.0
587,IGU_NEW,301100095,C,2023-01-23,2023-01-25,VL0161,"ALKALINDO ALAM ANUGRAH, PT",IID3ORG004,I-BEVERAGES,Lokal,...,BTL,IDR,0.0,IDR,1.0,4000.000,PPNM01,1.760000e+05,1.600000e+06,0.0
588,IGU_NEW,301100095,C,2023-01-23,2023-01-25,VL0161,"ALKALINDO ALAM ANUGRAH, PT",IID3ORG005,I-BEVERAGES,Lokal,...,BTL,IDR,0.0,IDR,1.0,4100.000,PPNM01,9.020000e+04,8.200000e+05,0.0
589,IGU_NEW,301100095,C,2023-01-23,2023-01-25,VL0161,"ALKALINDO ALAM ANUGRAH, PT",IID3ORG006,I-BEVERAGES,Lokal,...,BTL,IDR,0.0,IDR,1.0,8149.950,PPNM01,8.068450e+04,7.334955e+05,0.0


In [4]:
df.to_excel (r'PO Jan- jan 23.xlsx', index=True, header=True, merge_cells=True)